# Exploratorium 🐄👁️ Cow Eye Ball Cooking

### “Scientists and artists are the world's noticers. Their job is simply to notice what other people cannot.” —Frank Oppenheimer (1912–1985) founder of the Exploratorium


## 🔥 Let's get the required packages

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.11.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.11.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.11.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"
#r "nuget: SkiaSharp, 2.88.3"

In [2]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

## 🧑‍🍳 Let's get cooking on learning about cow eye dissection

### 🧠 There's an 8 page PDF available. This is the first two pages:

![](coweye2.jpg)
![](coweye3.jpg)

### 👁️ Let's set up a foundation model that can see/understand

In [ ]:
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.ChatCompletion;

const string ImageUri = "https://maeda.pm/wp-content/uploads/2024/05/coweye2.jpg";

const string OpenAIVisionEnabledModel = "gpt-4-vision-preview";

var visionKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(OpenAIVisionEnabledModel, apiKey)
    .Build();

var visionChatCompletionService = visionKernel.GetRequiredService<IChatCompletionService>();

var visionChatHistory = new ChatHistory(
    @"You are a friendly explainer with four years of experience working at 
    the Exploratorium with a background in biology or scientific communication.");

visionChatHistory.AddUserMessage(new ChatMessageContentItemCollection
{
    new TextContent(
        @"For this image, what are the key details to take away for a 
        person to explain how to dissect a cow's eye?"),
    new ImageContent(new Uri(ImageUri))
});

OpenAIPromptExecutionSettings settings = new() { MaxTokens = 500, Temperature = 0.1  };

var reply = await visionChatCompletionService.GetChatMessageContentAsync(visionChatHistory, settings);
Console.WriteLine(Utils.WordWrap(reply.Content, 80));

### 🧑‍💻 Let's do some abstraction of code to make things neater going forward

In [4]:
public async Task<string> AddVisionChatMessageAsync(string message)
{
    visionChatHistory.AddUserMessage(new ChatMessageContentItemCollection
    {
        new TextContent(message)
    });

    var reply = await visionChatCompletionService.GetChatMessageContentAsync(visionChatHistory, settings);
    Console.WriteLine(Utils.WordWrap(reply.Content, 80));
    return reply.Content;
}

public async Task<string> AddVisionChatMessageWithImageURLAsync(string message, string imageUrl)
{
    visionChatHistory.AddUserMessage(new ChatMessageContentItemCollection
    {
        new TextContent(message),
        new ImageContent(new Uri(imageUrl))
    });

    var reply = await visionChatCompletionService.GetChatMessageContentAsync(visionChatHistory, settings);
    Console.WriteLine(Utils.WordWrap(reply.Content, 80));
    return reply.Content;
}

### 🧑‍🎨 Let's get it to suggest how to prompt itself

In [ ]:
await AddVisionChatMessageAsync(
    @"For the images presented on the page, write image generation prompts 
    appropriate for recreating the images.");

### 🧠 Let's add the second page into this chat history

![](coweye3.jpg)

In [ ]:
string secondPageResult = await AddVisionChatMessageWithImageURLAsync(
    @"For this image, what are the key details to take away for a person 
    to explain how to dissect a cow's eye?",
    "https://maeda.pm/wp-content/uploads/2024/05/coweye3.jpg");

In [ ]:
string resultingPrompt = await AddVisionChatMessageAsync(
    @"For the first image presented on the page, 
    write an AI image generation prompt for DALL-E
    that will accurately recreate the image. Make it 
    as detailed as possible.");

### 🧑‍🎨 Let's flex our image generation muscles a bit

In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0002, SKEXP0012

using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.TextToImage;

Kernel painterKernel = Kernel.CreateBuilder()
    .AddOpenAITextToImage(apiKey) // Add your text to image service
    .Build();

ITextToImageService dallE = painterKernel.GetRequiredService<ITextToImageService>();

var imgPrompt = resultingPrompt;

imgPrompt = @"Create an image of a persons hands holding a cow's eye";

var image = await dallE.GenerateImageAsync(
    imgPrompt,
    256, 256);

Console.WriteLine(image);


In [ ]:
using System.IO;
using System.Net.Http;
#!import ../config/SkiaUtils.cs

HttpClient httpClient = new HttpClient();

async Task DownloadImageAsync(string uri, string localPath)
{
    using (HttpResponseMessage response = await httpClient.GetAsync(uri))
    {
        response.EnsureSuccessStatusCode();

        using (Stream contentStream = await response.Content.ReadAsStreamAsync(), 
                fileStream = new FileStream(localPath, FileMode.Create, FileAccess.Write, FileShare.None))
        {
            await contentStream.CopyToAsync(fileStream);
        }
    }
}

async Task MyShowImage(string uri)
{
    string uniqueFileName = "abc-" + Guid.NewGuid().ToString() + ".png";
    await DownloadImageAsync(uri, uniqueFileName);
    await SkiaUtils.ShowImage(image, 256, 256);
}

MyShowImage(image).Wait();

### 🙉 Let's add the auditory modalities

In [ ]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/Utils.cs 
#!import ../config/AzureSpeech.cs

//Utils.LoadEnvFile();

string subscriptionKey = Environment.GetEnvironmentVariable("AZURE_SPEECH_KEY");
string subscriptionRegion = Environment.GetEnvironmentVariable("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

### 🎤 Let's listen up

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

### 🗣️ And let's speak up

In [ ]:
#pragma warning disable SKEXP0101

await speechService.SynthesizeSpeechAsync(theAnswer);

## 🧑‍🎓 Let's get an inquisitive 6th grader here

In [ ]:
var chatKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(model, apiKey)
    .Build();

var systemMessage =  
  @"You are a 6th grader visiting the world-famous Exploratorium in San Francisco, 
  founded by Frank Oppenheimer. Today, you are observing a cow eye dissection 
  demonstrated by one of the Exploratorium's professional science communicators. 
  You are curious about the dissection and have many questions, but 
  you have no prior knowledge about cow eyes. You are here to learn 
  from the explainer, not to teach. Please keep your questions concise 
  and limited to one sentence each. When asked a question, respond in the
  manner of a sixth grader with little to no prior knowledge about cow eyes.
    ";

var chatHistory = new ChatHistory( systemMessage );
var chatCompletions = chatKernel.GetRequiredService<IChatCompletionService>();

string userMessage;

userMessage = @"Hi, I'm an expert explainer of the cow eye dissection. 
    I can help you with any questions you have.";

while(true) {
    Console.WriteLine(Utils.WordWrap("Explainer: " + userMessage, 80));

    chatHistory.AddUserMessage(userMessage);

    var assistantReply = 
        await chatCompletions.GetChatMessageContentAsync(
            chatHistory, 
            new OpenAIPromptExecutionSettings());

    Console.WriteLine(Utils.WordWrap("AI Learner: " + assistantReply.Content, 80));
    await speechService.SynthesizeSpeechAsync(assistantReply.Content);

    userMessage = await InteractiveKernel.GetInputAsync("Enter 'bye' to end:");

    if (userMessage.ToLower() == "bye") {
        break;
    }
}

## 🧑‍🔬 Let's get an expert Explainer involved

You can easily make a GPT instead of building this by hand ...

![](coweyegpt.png)

But we like to build stuff at the Exploratorium so ...

In [ ]:
var chatKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(model, apiKey)
    .Build();

var systemMessage =  
@"You are an expert explainer at the Exploratorium, 
ready to guide a 6th grader through the exciting process of 
cow eye dissection. Your task is to teach them step-by-step, 
using simple language. Start by discussing the importance of safety, 
like careful handling of the scalpel and the importance of hygiene 
after touching raw materials.

Explain the external parts of the eye such as the sclera and cornea, 
and then demonstrate making an incision in the cornea to release the 
aqueous humor. Guide the student through cutting the eye in half 
to expose the internal structures like the lens and retina, 
and discuss their functions.

Make sure to cover the removal of the lens and the exploration 
of the vitreous humor. Highlight the interesting features of the cow's eye, 
such as the tapetum, which helps in night vision. Your explanations should 
be concise and engaging, designed to spark curiosity and deepen 
understanding of eye anatomy.

Always encourage questions and interact in a supportive manner 
to create a memorable educational experience for the student.

You're talking with elementary school students, so keep the langauge
simple and easy to understand. Also keep your responses extremely short so that 
the learner can ask questions and engage with the material.
";

var chatHistory = new ChatHistory( systemMessage );
var chatCompletions = chatKernel.GetRequiredService<IChatCompletionService>();

string userMessage;

userMessage = @"Hi, I'm a 6th grader and want to learn more about a cow's eye.";

while(true) {
    Console.WriteLine(Utils.WordWrap("Learner: " + userMessage, 80));

    chatHistory.AddUserMessage(userMessage);

    var assistantReply = 
        await chatCompletions.GetChatMessageContentAsync(
            chatHistory, 
            new OpenAIPromptExecutionSettings());

    Console.WriteLine(Utils.WordWrap("AI Explainer: " + assistantReply.Content, 80));
    await speechService.SynthesizeSpeechAsync(assistantReply.Content);

    userMessage = await InteractiveKernel.GetInputAsync("Enter 'bye' to end:");

    if (userMessage.ToLower() == "bye") {
        break;
    }
}